In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import sys

#--------------
#   QUERY 3
#--------------

#The chances of winning given the first hand
#It helps to maximize the profit and minimize the loss by evaluating the initial hand

spark = SparkSession.builder \
    .appName("TGVD_GenericQuery") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

path_training = "CardsParquetData/trained_blackjack.parquet"
path_match = "CardsParquetData/played_blackjack.parquet"

df_train = spark.read.parquet(path_training)
df_play = spark.read.parquet(path_match)

df_play.show()
df_train.show()

+-------------------+-----------+--------+--------+--------+--------+--------+------+
|          Timestamp|Shown_cards|  Hand 0|  Hand 1|  Hand 2|  Hand 3|  Hand 4|Hand 5|
+-------------------+-----------+--------+--------+--------+--------+--------+------+
|2025-05-12 19:25:11|    [10, 1]| [14, 3]|[19, 15]|[19, 21]|[19, 21]|    NULL|  NULL|
|2025-05-12 19:25:11|     [4, 5]| [8, 13]|[15, 15]|[15, 26]|    NULL|    NULL|  NULL|
|2025-05-12 19:25:11|    [12, 3]|[12, 14]|[12, 19]|[12, 15]|[12, 18]|[12, 26]|  NULL|
|2025-05-12 19:25:11|   [11, 12]|[21, 20]|[21, 20]|    NULL|    NULL|    NULL|  NULL|
|2025-05-12 19:25:11|    [10, 3]|[10, 12]|[10, 14]|[10, 17]|[10, 22]|    NULL|  NULL|
|2025-05-12 19:25:11|   [10, 10]|[10, 18]|[10, 30]|    NULL|    NULL|    NULL|  NULL|
|2025-05-12 19:25:11|     [8, 4]|[19, 14]|[19, 14]|    NULL|    NULL|    NULL|  NULL|
|2025-05-12 19:25:11|    [11, 9]|[19, 21]|[19, 21]|    NULL|    NULL|    NULL|  NULL|
|2025-05-12 19:25:11|     [7, 3]|[15, 13]|[15, 24]|   

In [66]:
from pyspark.sql.functions import floor
from pyspark.sql.functions import monotonically_increasing_id

df_train = df_train.withColumn("index", (monotonically_increasing_id() + 1))
df_play = df_play.withColumn("index", (monotonically_increasing_id() + 1))

CHUNK_SIZE = 440
df_chunks_train = df_train.withColumn("Chunk Number", floor(col("index")/CHUNK_SIZE))
df_chunks_play = df_play.withColumn("Chunk Number", floor(col("index")/CHUNK_SIZE))

df_chunks_play.show()

+-------------------+-----------+--------+--------+--------+--------+--------+------+-----+------------+
|          Timestamp|Shown_cards|  Hand 0|  Hand 1|  Hand 2|  Hand 3|  Hand 4|Hand 5|index|Chunk Number|
+-------------------+-----------+--------+--------+--------+--------+--------+------+-----+------------+
|2025-05-12 19:25:11|    [10, 1]| [14, 3]|[19, 15]|[19, 21]|[19, 21]|    NULL|  NULL|    1|           0|
|2025-05-12 19:25:11|     [4, 5]| [8, 13]|[15, 15]|[15, 26]|    NULL|    NULL|  NULL|    2|           0|
|2025-05-12 19:25:11|    [12, 3]|[12, 14]|[12, 19]|[12, 15]|[12, 18]|[12, 26]|  NULL|    3|           0|
|2025-05-12 19:25:11|   [11, 12]|[21, 20]|[21, 20]|    NULL|    NULL|    NULL|  NULL|    4|           0|
|2025-05-12 19:25:11|    [10, 3]|[10, 12]|[10, 14]|[10, 17]|[10, 22]|    NULL|  NULL|    5|           0|
|2025-05-12 19:25:11|   [10, 10]|[10, 18]|[10, 30]|    NULL|    NULL|    NULL|  NULL|    6|           0|
|2025-05-12 19:25:11|     [8, 4]|[19, 14]|[19, 14]|    

In [67]:
#Funtion to detect if we have a win, draw or a lose
from pyspark.sql.functions import coalesce, when

rev_hand_cols = ["Hand 7", "Hand 6", "Hand 5", "Hand 4", "Hand 3", "Hand 2", "Hand 1", "Hand 0"]
df_result_train = df_chunks_train.withColumn("Final_Hand", coalesce(*[col(c) for c in rev_hand_cols]))
df_result_train.show()

df_result_train = df_result_train.withColumn(
    "Result",
    when(col("Final_Hand").isNull(), "Unknown")
    .when(col("Final_Hand")[0] > 21, "Lose")
    .when(col("Final_Hand")[1] > 21, "Win")
    .when(col("Final_Hand")[0] > col("Final_Hand")[1], "Win")
    .when(col("Final_Hand")[0] < col("Final_Hand")[1], "Lose")
    .otherwise("Draw")
)
df_result_train.show()

rev_hand_cols = ["Hand 5", "Hand 4", "Hand 3", "Hand 2", "Hand 1", "Hand 0"]
df_result_play = df_chunks_play.withColumn("Final_Hand", coalesce(*[col(c) for c in rev_hand_cols]))
df_result_play.show()

df_result_play = df_result_play.withColumn(
    "Result",
    when(col("Final_Hand").isNull(), "Unknown")
    .when(col("Final_Hand")[0] > 21, "Lose")
    .when(col("Final_Hand")[1] > 21, "Win")
    .when(col("Final_Hand")[0] > col("Final_Hand")[1], "Win")
    .when(col("Final_Hand")[0] < col("Final_Hand")[1], "Lose")
    .otherwise("Draw")
)
df_result_play.show()

+-------------------+-----------+--------+--------+--------+--------+------+------+------+------+-----+------------+----------+
|          Timestamp|Shown_cards|  Hand 0|  Hand 1|  Hand 2|  Hand 3|Hand 4|Hand 5|Hand 6|Hand 7|index|Chunk Number|Final_Hand|
+-------------------+-----------+--------+--------+--------+--------+------+------+------+------+-----+------------+----------+
|2025-05-12 19:24:20|    [10, 4]| [10, 7]|[10, 11]|[10, 21]|[10, 21]|  NULL|  NULL|  NULL|  NULL|    1|           0|  [10, 21]|
|2025-05-12 19:24:20|     [4, 7]|[13, 12]|[23, 21]|    NULL|    NULL|  NULL|  NULL|  NULL|  NULL|    2|           0|  [23, 21]|
|2025-05-12 19:24:20|   [-1, 10]|[14, 18]|[14, 29]|    NULL|    NULL|  NULL|  NULL|  NULL|  NULL|    3|           0|  [14, 29]|
|2025-05-12 19:24:20|    [8, 12]|[20, 12]|[22, 12]|    NULL|    NULL|  NULL|  NULL|  NULL|  NULL|    4|           0|  [22, 12]|
|2025-05-12 19:24:20|     [3, 6]| [3, 15]| [3, 23]|    NULL|    NULL|  NULL|  NULL|  NULL|  NULL|    5| 

In [68]:
df_clean_play = df_result_play.withColumn("1st Hand Card", col("Shown_cards")[0]).select("Chunk Number", "Final_Hand", "Result", "1st Hand Card")
df_clean_play.show()
df_clean_train = df_result_play.withColumn("1st Hand Card", col("Shown_cards")[0]).select("Chunk Number", "Final_Hand", "Result", "1st Hand Card")
df_clean_train.show()

+------------+----------+------+-------------+
|Chunk Number|Final_Hand|Result|1st Hand Card|
+------------+----------+------+-------------+
|           0|  [19, 21]|  Lose|           10|
|           0|  [15, 26]|   Win|            4|
|           0|  [12, 26]|   Win|           12|
|           0|  [21, 20]|   Win|           11|
|           0|  [10, 22]|   Win|           10|
|           0|  [10, 30]|   Win|           10|
|           0|  [19, 14]|   Win|            8|
|           0|  [19, 21]|  Lose|           11|
|           0|  [15, 24]|   Win|            7|
|           0|  [12, 28]|   Win|            8|
|           0|  [12, 27]|   Win|            6|
|           0|  [22, 18]|  Lose|            8|
|           0|  [20, 10]|   Win|            8|
|           0|  [20, 18]|   Win|            5|
|           0|  [26, 17]|  Lose|            8|
|           0|  [11, 26]|   Win|            6|
|           0|  [18, 25]|   Win|            5|
|           0|  [17, 25]|   Win|            5|
|           0

In [69]:
df_clean_play = df_clean_play.groupBy("1st Hand Card", "Result", "Chunk Number").count().orderBy(["Chunk Number", "count", "1st Hand Card"], ascending = False)
df_clean_play.show()

df_clean_train = df_clean_train.groupBy("1st Hand Card", "Result", "Chunk Number").count().orderBy(["Chunk Number", "count", "1st Hand Card"], ascending = False)
df_clean_train.show()

+-------------+------+------------+-----+
|1st Hand Card|Result|Chunk Number|count|
+-------------+------+------------+-----+
|            8|   Win|           2|   10|
|           10|   Win|           2|    9|
|           11|   Win|           2|    7|
|            8|  Lose|           2|    6|
|            6|  Lose|           2|    6|
|           12|  Lose|           2|    5|
|           11|  Lose|           2|    5|
|            6|   Win|           2|    5|
|            4|   Win|           2|    5|
|            3|   Win|           2|    5|
|            3|  Lose|           2|    5|
|           12|   Win|           2|    4|
|           10|  Lose|           2|    4|
|            7|  Lose|           2|    4|
|            5|   Win|           2|    4|
|            5|  Lose|           2|    4|
|            4|  Lose|           2|    4|
|            2|  Lose|           2|    4|
|            7|   Win|           2|    3|
|            2|   Win|           2|    3|
+-------------+------+------------

In [72]:
from pyspark.sql.functions import sum, count

total_df = df_clean_play.groupBy("1st Hand Card", "Chunk Number").agg(count("*").alias("TotalGames"))
wins_df = df_clean_play.filter(col("Result") == "Win").groupBy("1st Hand Card", "Chunk Number").agg(count("*").alias("Wins"))

df_winrate = total_df.join(wins_df, on=["1st Hand Card", "Chunk Number"], how="left").fillna(0, subset=["Wins"]).withColumn("Winning Rate Proportion", col("Wins") / col("TotalGames"))
df_winrate.orderBy("Winning Rate Proportion", ascending = False).show()

+-------------+------------+----------+----+-----------------------+
|1st Hand Card|Chunk Number|TotalGames|Wins|Winning Rate Proportion|
+-------------+------------+----------+----+-----------------------+
|           -1|           2|         2|   1|                    0.5|
|            1|           0|         2|   1|                    0.5|
|            1|           1|         2|   1|                    0.5|
|            0|           1|         2|   1|                    0.5|
|           11|           1|         2|   1|                    0.5|
|           10|           1|         2|   1|                    0.5|
|            3|           2|         2|   1|                    0.5|
|            2|           2|         2|   1|                    0.5|
|           -1|           1|         2|   1|                    0.5|
|            2|           0|         2|   1|                    0.5|
|            6|           2|         2|   1|                    0.5|
|            0|           0|      